In [8]:
import torch
import numpy as np
import xgboost as xgb
from onnxconverter_common.data_types import FloatTensorType
from hummingbird import convert_sklearn

# Create some random data for binary classification
num_classes = 2
X = np.array(np.random.rand(100000, 28), dtype=np.float32)
y = np.random.randint(num_classes, size=100000)

In [9]:
# Create and train a model (XGB in this case)
model = xgb.XGBClassifier()
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [10]:
# Use Hummingbird to convert the model to pytorch
pytorch_model = convert_sklearn(model,[("input", FloatTensorType([100000, 28]))])

In [11]:
%%timeit -r 3

# Run Hummingbird on CPU
pytorch_model.to('cpu')
hum_cpu = pytorch_model(torch.from_numpy(X))

638 ms ± 1.58 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [12]:
%%timeit -r 3

# Run hummingbird on GPU
pytorch_model.to('cuda')
hum_gpu = pytorch_model(torch.from_numpy(X).to('cuda'))


12.4 ms ± 26.3 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)
